In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold, LeaveOneGroupOut
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler, LabelEncoder
from imblearn.over_sampling import SMOTE
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, Concatenate, TimeDistributed
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tqdm import tqdm
import time
import warnings

# Change csv files to dataframes
def read_csv_files(directory):
    data_frames = {}
    print("\nReading CSV files:")
    for filename in os.listdir(directory):
        if filename.endswith('.csv'):
            print(f"  Reading: {filename}")
            df = pd.read_csv(os.path.join(directory, filename))
            parts = filename.split('_')
            file_type = parts[0]
            participant = parts[-2] if len(parts) > 2 else 'unknown'
            difficulty = parts[-1].split('.')[0] if len(parts) > 1 else 'unknown'
            if not file_type == 'E4 ACC data':
                key = (participant, difficulty, file_type)
                data_frames[key] = df
            else:
                print(f"    Skipped: {filename} (ACC data)")
    print(f"Total files read: {len(data_frames)}")
    return data_frames

# Select useful data only (after game start, interpolate)
def preprocess_data(df, baseline_duration):
    if 'started' in df['game started'].values:
        start_idx = df[df['game started'] == 'started'].index[0]
        start_unix_time = df.loc[start_idx, 'UNIX Time']
        target_unix_time = start_unix_time - baseline_duration
        idx = (df['UNIX Time'] - target_unix_time).abs().idxmin()
        df = df.iloc[idx:]
    else:
        print("Warning: 'game start' marker not found. Using all data.")
        
    warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)
    numeric_columns = df.select_dtypes(include=[np.number]).columns
    df[numeric_columns] = df[numeric_columns].interpolate(method='linear', limit_direction='both')
    
    return df

def z_score_normalize(df, baseline_duration):
    numeric_columns = df.select_dtypes(include=[np.number]).columns.drop('UNIX Time')
    baseline = df.iloc[:baseline_duration]
    mean = baseline[numeric_columns].mean()
    std = baseline[numeric_columns].std()
    std = std.replace(0, 1)  # Replace zero std with 1 to avoid division by zero
    normalized_df = df.copy()
    normalized_df[numeric_columns] = (df[numeric_columns] - mean) / std
    return normalized_df

def process_data(df, baseline_duration):
    df = preprocess_data(df, baseline_duration)
    
    if 'UNIX Time' not in df.columns or df['UNIX Time'].isnull().all():
        print("Error: 'UNIX Time' column not found or all values are null")
        return pd.DataFrame()
    
    game_start_time = df['UNIX Time'].min() + 10
    
    unix_time = df['UNIX Time']
    df_numeric = df.select_dtypes(include=[np.number]).drop(columns=['UNIX Time'])
    df_numeric['UNIX Time'] = unix_time
    
    normalized_df = z_score_normalize(df_numeric, baseline_duration)
    
    df_after_start = normalized_df[normalized_df['UNIX Time'] >= game_start_time]
    
    if df_after_start.empty:
        print(f"Warning: No data available after assumed game start")
        return pd.DataFrame()
    
    df_numeric = df_after_start.drop(columns=['UNIX Time'])
    
    return df_numeric

def make_sequences(df, window_size):
    sequences = []
    for i in range(0, len(df) - window_size + 1):
        sequences.append(df.iloc[i:i+window_size].values)
    return np.array(sequences)

def create_dataset(data_frames, window_size, expected_features, baseline_duration):
    X_data = []
    y_data = []
    participant_data = []
    all_columns = set()
    
    for (participant, difficulty, file_type), df in data_frames.items():
        processed_df = process_data(df, baseline_duration)
        all_columns.update(processed_df.columns)
    
    all_columns = sorted(list(all_columns))[:expected_features]  # Limit to expected number of features
    
    for (participant, difficulty, file_type), df in data_frames.items():
        processed_df = process_data(df, baseline_duration)
        
        for col in all_columns:
            if col not in processed_df.columns:
                processed_df[col] = 0
        
        processed_df = processed_df[all_columns]
        
        # Pad or truncate to match expected features
        if processed_df.shape[1] < expected_features:
            padding = pd.DataFrame(0, index=processed_df.index, 
                                   columns=[f'pad_{i}' for i in range(expected_features - processed_df.shape[1])])
            processed_df = pd.concat([processed_df, padding], axis=1)
        elif processed_df.shape[1] > expected_features:
            processed_df = processed_df.iloc[:, :expected_features]
            
        sequences = make_sequences(processed_df, window_size)
        if len(sequences) > 0:
            X_data.append(sequences)
            y_data.extend([difficulty] * len(sequences))
            participant_data.extend([participant] * len(sequences))
    
    X = np.concatenate(X_data)
    y = np.array(y_data)
    participants = np.array(participant_data)
    
    return X, y, participants

def create_advanced_lstm_model(input_shape, n_classes, lstm_units, dense_units, dropout_rate):
    inputs = Input(shape=input_shape)
    
    low_level = TimeDistributed(Dense(32, activation='relu'))(inputs)
    low_level = TimeDistributed(Dense(16, activation='relu'))(low_level)
    
    high_level = LSTM(lstm_units[0], return_sequences=True)(inputs)
    high_level = LSTM(lstm_units[1])(high_level)
    
    combined = Concatenate()([tf.keras.layers.Flatten()(low_level), high_level])
    
    x = Dense(dense_units, activation='relu')(combined)
    x = Dropout(dropout_rate)(x)
    
    outputs = Dense(n_classes, activation='softmax')(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

def loso_evaluation(X, y, participants, n_classes, hyperparameters):
    logo = LeaveOneGroupOut()
    
    loso_scores = []
    for i, (train_index, test_index) in enumerate(logo.split(X, y, participants)):
        print(f"\nLOSO Iteration {i+1}/{len(np.unique(participants))}")
        print(f"Test participant: {participants[test_index[0]]}")
        
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        # Validation split from the train set
        val_split_idx = int(0.8 * len(X_train))  # 80% train, 20% validation
        X_train, X_val = X_train[:val_split_idx], X_train[val_split_idx:]
        y_train, y_val = y_train[:val_split_idx], y_train[val_split_idx:]
        
        model = create_advanced_lstm_model(
            input_shape=(X_train.shape[1], X_train.shape[2]),
            n_classes=n_classes,
            lstm_units=hyperparameters['lstm_units'],
            dense_units=hyperparameters['dense_units'],
            dropout_rate=hyperparameters['dropout_rate']
        )
        
        le = LabelEncoder()
        y_train_cat = to_categorical(le.fit_transform(y_train))
        y_val_cat = to_categorical(le.transform(y_val))
        y_test_cat = to_categorical(le.transform(y_test))
        
        # EarlyStopping 콜백 추가
        early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
        
        print("Training model...")
        history = model.fit(X_train, 
                            y_train_cat,
                            epochs=hyperparameters['epochs'], 
                            batch_size=hyperparameters['batch_size'],
                            validation_data=(X_val, y_val_cat),
                            callbacks=[early_stopping], 
                            verbose=0)
        
        # 학습 그래프 그리기
        plt.figure(figsize=(12, 5))

        # Loss 그래프
        plt.subplot(1, 2, 1)
        plt.plot(history.history['loss'], label='Training Loss')
        plt.plot(history.history['val_loss'], label='Validation Loss')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.title('Training and Validation Loss')
        plt.legend()

        # Accuracy 그래프
        plt.subplot(1, 2, 2)
        plt.plot(history.history['accuracy'], label='Training Accuracy')
        plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')
        plt.title('Training and Validation Accuracy')
        plt.legend()

        plt.tight_layout()
        plt.show()
        
        print("Evaluating model...")
        y_pred = model.predict(X_test)
        y_pred_classes = np.argmax(y_pred, axis=1)
        y_test_classes = np.argmax(y_test_cat, axis=1)
        
        accuracy = accuracy_score(y_test_classes, y_pred_classes)
        loso_scores.append(accuracy)
        print(f"Accuracy for participant {participants[test_index[0]]}: {accuracy:.4f}")
        
        print("Classification Report:")
        print(classification_report(y_test_classes, y_pred_classes, target_names=le.classes_))
        
        cm = confusion_matrix(y_test_classes, y_pred_classes)
        disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=le.classes_)
        disp.plot(cmap=plt.cm.Blues)
        plt.title(f'Confusion Matrix for participant {participants[test_index[0]]}')
        plt.show()
    
    return np.mean(loso_scores), np.std(loso_scores)

def stratified_kfold_evaluation(X, y, n_classes, hyperparameters, label_encoder):
    skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    
    fold_accuracies = []
    
    for i, (train_index, test_index) in enumerate(skf.split(X, y)):
        print(f"\nStratified K-Fold Iteration {i+1}/{skf.n_splits}")
        
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        # Apply SMOTE only on the training data
        print("Applying SMOTE to the training data...")
        smote = SMOTE(random_state=42)
        X_train_flat = X_train.reshape(X_train.shape[0], -1)
        X_train_resampled, y_train_resampled = smote.fit_resample(X_train_flat, y_train)
        X_train_resampled = X_train_resampled.reshape(-1, X_train.shape[1], X_train.shape[2])
        print(f"After SMOTE - Training data shape: {X_train_resampled.shape}, Training labels shape: {y_train_resampled.shape}")
        
        # Encode labels (using the pre-fitted encoder)
        y_train_cat = to_categorical(label_encoder.transform(y_train_resampled), num_classes=n_classes)
        y_test_cat = to_categorical(label_encoder.transform(y_test), num_classes=n_classes)
        
        # EarlyStopping callback
        early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
        
        # Create and train the model
        model = create_advanced_lstm_model(
            input_shape=(X_train_resampled.shape[1], X_train_resampled.shape[2]),
            n_classes=n_classes,
            lstm_units=hyperparameters['lstm_units'],
            dense_units=hyperparameters['dense_units'],
            dropout_rate=hyperparameters['dropout_rate']
        )
        
        print("Training model...")
        history = model.fit(
            X_train_resampled, y_train_cat,
            epochs=hyperparameters['epochs'],
            batch_size=hyperparameters['batch_size'],
            validation_data=(X_test, y_test_cat),
            callbacks=[early_stopping],
            verbose=1  # Set to 1 for more detailed output
        )
        
        # Plot training history
        plt.figure(figsize=(12, 5))

        # Loss plot
        plt.subplot(1, 2, 1)
        plt.plot(history.history['loss'], label='Training Loss')
        plt.plot(history.history['val_loss'], label='Validation Loss')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.title(f'Fold {i+1} - Training and Validation Loss')
        plt.legend()

        # Accuracy plot
        plt.subplot(1, 2, 2)
        plt.plot(history.history['accuracy'], label='Training Accuracy')
        plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')
        plt.title(f'Fold {i+1} - Training and Validation Accuracy')
        plt.legend()

        plt.tight_layout()
        plt.show()
        
        # Evaluate the model
        print("Evaluating model on validation data...")
        y_pred = model.predict(X_test)
        y_pred_classes = np.argmax(y_pred, axis=1)
        y_test_classes = np.argmax(y_test_cat, axis=1)
        
        accuracy = accuracy_score(y_test_classes, y_pred_classes)
        fold_accuracies.append(accuracy)
        print(f"Accuracy for fold {i+1}: {accuracy:.4f}")
        
        print("Classification Report:")
        print(classification_report(y_test_classes, y_pred_classes, target_names=label_encoder.classes_))
        
        cm = confusion_matrix(y_test_classes, y_pred_classes)
        disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=label_encoder.classes_)
        disp.plot(cmap=plt.cm.Blues)
        plt.title(f'Confusion Matrix for fold {i+1}')
        plt.show()
    
    return np.mean(fold_accuracies), np.std(fold_accuracies)

def main():
    directory = r'C:\Users\USER\Downloads\pilotdata'
    data_frames = read_csv_files(directory)
    
    window_size = 10  # 10-second window
    n_classes = 5  # Only 5-class classification
    expected_features = 11  # Set this to the number of features you expect
    baseline_duration = 30  # Set this to your baseline duration in seconds
    
    print(f"\n{'='*50}")
    print(f"Running 5-class classification with Stratified K-Fold evaluation")
    print(f"{'='*50}")
    
    X, y, participants = create_dataset(data_frames, window_size, expected_features, baseline_duration)
    
    if X is None or y is None or participants is None:
        print("Failed to create dataset.")
        return
    
    print(f"Dataset shape: {X.shape}")
    print(f"Number of sequences: {X.shape[0]}")
    print(f"Sequence length: {X.shape[1]}")
    print(f"Number of features: {X.shape[2]}")
    print(f"Class distribution: {np.unique(y, return_counts=True)}")
    
    # Encode labels once
    le = LabelEncoder()
    le.fit(y)  # Fit on the entire dataset to ensure consistency
    y_encoded = le.transform(y)
    
    print("\nAfter Label Encoding:")
    print(f"Classes: {le.classes_}")
    print(f"Encoded labels: {np.unique(y_encoded, return_counts=True)}")
    
    # Use the best hyperparameters you provided
    best_hyperparameters = {
        'lstm_units': (32, 16),
        'dense_units': 64,
        'dropout_rate': 0.3,
        'epochs': 50,
        'batch_size': 256
    }
    
    try:
        print("\nStarting Stratified K-Fold evaluation with best hyperparameters...")
        loso_mean, loso_std = stratified_kfold_evaluation(X, y_encoded, n_classes, best_hyperparameters, le)
        print(f"\nStratified K-Fold evaluation results:")
        print(f"Mean accuracy: {loso_mean:.4f} (+/- {loso_std:.4f})")
        
        # Optionally, train a final model on the entire dataset
        # Apply SMOTE to the entire dataset (if desired), but be cautious of data leakage
        # Here, we'll skip SMOTE for the final model to avoid potential leakage
        final_model = create_advanced_lstm_model(
            input_shape=(X.shape[1], X.shape[2]),
            n_classes=n_classes,
            lstm_units=best_hyperparameters['lstm_units'],
            dense_units=best_hyperparameters['dense_units'],
            dropout_rate=best_hyperparameters['dropout_rate']
        )
        
        y_cat = to_categorical(y_encoded, num_classes=n_classes)
        
        print("\nTraining final model on all data...")
        early_stopping_final = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)
        final_model.fit(X, y_cat, epochs=best_hyperparameters['epochs'], 
                       batch_size=best_hyperparameters['batch_size'], 
                       callbacks=[early_stopping_final], verbose=1)
        
        # Save the final model
        model_filename = r"C:\Users\HCIS\Downloads\pilotdata\best_lstm_model_lowhigh_3class0723.h5"
        final_model.save(model_filename)
        print(f"\nFinal model saved as {model_filename}")
        
        # Save the label encoder
        le_filename = r"C:\Users\HCIS\Downloads\pilotdata\label_encoder_lowhigh0723.joblib"
        joblib.dump(le, le_filename)
        print(f"Label encoder saved as {le_filename}")
        
    except Exception as e:
        import traceback
        print(f"An error occurred during the kfold evaluation or model saving:")
        print(str(e))
        print("Traceback:")
        traceback.print_exc()


if __name__ == "__main__":
    main()


Reading CSV files:
  Reading: differential_cy_easier.csv
  Reading: differential_cy_easy.csv
  Reading: differential_cy_hard.csv
  Reading: differential_cy_harder.csv
  Reading: differential_cy_moderate.csv
  Reading: differential_dh_easier.csv
  Reading: differential_dh_easy.csv
  Reading: differential_dh_hard.csv
  Reading: differential_dh_harder.csv
  Reading: differential_dh_moderate.csv
  Reading: differential_gb_easier.csv
  Reading: differential_gb_easy.csv
  Reading: differential_gb_hard.csv
  Reading: differential_gb_harder.csv
  Reading: differential_gb_moderate.csv
  Reading: differential_jj_easier.csv
  Reading: differential_jj_easy.csv
  Reading: differential_jj_hard.csv
  Reading: differential_jj_harder.csv
  Reading: differential_jj_moderate.csv
  Reading: differential_jw_easier.csv
  Reading: differential_jw_easy.csv
  Reading: differential_jw_hard.csv
  Reading: differential_jw_harder.csv
  Reading: differential_jw_moderate.csv
  Reading: differential_kb_easier.csv
  

Traceback (most recent call last):
  File "c:\Users\USER\anaconda3\lib\site-packages\sklearn\utils\_encode.py", line 225, in _encode
    return _map_to_integer(values, uniques)
  File "c:\Users\USER\anaconda3\lib\site-packages\sklearn\utils\_encode.py", line 165, in _map_to_integer
    return np.array([table[v] for v in values])
  File "c:\Users\USER\anaconda3\lib\site-packages\sklearn\utils\_encode.py", line 165, in <listcomp>
    return np.array([table[v] for v in values])
  File "c:\Users\USER\anaconda3\lib\site-packages\sklearn\utils\_encode.py", line 159, in __missing__
    raise KeyError(key)
KeyError: '0'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\USER\AppData\Local\Temp\ipykernel_8372\143342927.py", line 379, in main
    loso_mean, loso_std = stratified_kfold_evaluation(X, y_encoded, participants, n_classes, best_hyperparameters, le)
  File "C:\Users\USER\AppData\Local\Temp\ipykernel_8372\143342927.p